In [1]:
import os,sys
sys.path.append(os.getcwd())

#Module imports
import geeViz.getImagesLib as getImagesLib
ee = getImagesLib.ee
Map = getImagesLib.Map

print('done')

/Users/jessieeastburn/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Initializing GEE
Successfully initialized
geeViz package folder: /Users/jessieeastburn/Library/Python/3.9/lib/python/site-packages/geeViz
done


In [2]:
import pandas as pd
import time
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

/var/folders/sf/r2b9cx3s6fl_d5hscrs0_44c0000gn/T/ipykernel_97938/518313686.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
# Get utka polygon.
studyArea = (
    ee.FeatureCollection('projects/disco-glass-413820/assets/plot_min_bound_geom')
    .filter("site_id == 'UOFU_UTKA'")
    .first()
    .geometry()
)

Map.port = 1992
Map.clearMap()

studyArea = ee.Geometry.Polygon(
            [[[-112.2647, 37.02876], 
              [-112.2647, 37.10894],
              [-112.0867, 37.10894],
              [-112.0867, 37.02876], 
              [-112.2647, 37.02876]]])
Map.centerObject(studyArea, 11)
Map.addLayer(studyArea, {'color': 'blue'}, 'utka')
Map.view()


Adding layer: utka
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1992/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://localhost:1992/geeView/?projectID=None&accessToken=ya29.a0AfB_byD8vaCIJorViR0m53PrxDtT2A5uHHYDCDy5nin9iugFs5dY5mKxTrQLTjjY8_qH7lwL6IEJ7_ffM4SzXiZoGgGFBYr63FKMFQBFwavPXCC6Y2SxIExittWoVFfZzgx2g_EkBjihPHJRVOXJ8aQ03MrbIeb0pkFOegaCgYKAWUSARISFQHGX2MiV09TnLjR_C8aSheIbz-Q1Q0173


::1 - - [21/Feb/2024 15:03:20] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byD8vaCIJorViR0m53PrxDtT2A5uHHYDCDy5nin9iugFs5dY5mKxTrQLTjjY8_qH7lwL6IEJ7_ffM4SzXiZoGgGFBYr63FKMFQBFwavPXCC6Y2SxIExittWoVFfZzgx2g_EkBjihPHJRVOXJ8aQ03MrbIeb0pkFOegaCgYKAWUSARISFQHGX2MiV09TnLjR_C8aSheIbz-Q1Q0173 HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/js/load.min.js HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/styles/style.min.css HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/js/gena-gee-palettes.js HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/js/lcms-viewer.min.js HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/assets/images/GEE_logo_transparent.png HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/assets/images/usfslogo.png HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/assets/images

In [7]:
# Run get images, set parameters manually, and export
lsAndTs = getImagesLib.getLandsatWrapper(\

    # Study area and time
    studyArea = studyArea,
    startYear = 2015,
    endYear = 2020,
    startJulian = 1,
    endJulian = 365,

    # Moving window options
    timebuffer =  0,# If multi-year moving window is needed, use timebuffer > 0 (e.g. timebuffer = 1 will result in a 3 year moving window)
    weights =  [1],# To compliment the timebuffer, set weights for each year of the moving window. This enables the center year to have more weight (e.g. [1,3,1] would weight the center year if timebuffer = 1 3x over the buffer years)

    # Composite methods and input imagery
    compositingMethod = 'medoid', # Specify median or medoid. Medoid allows for storing only the spectral and ancillary bands without indices
    toaOrSR = 'SR', # It is advised to only use TOA when combining Landsat and Sentinel-2 since the Sentinel-2 SR data in GEE are terrain-corrected and Landsat are not
    includeSLCOffL7 = False, # Whether to include Landsat 7 after May 2003. Scanline artifacts can be found in composites when True
    defringeL5 = True, # Whether to get rid of fringe edges found mostly on Landsat 4 and 5
    landsatCollectionVersion = 'C2', # To use Collection 1 or 2. Only use C1 if doing legacy work prior to 2022

    # Cloud and shadow masking
    # There will be a parameter for applying CloudScore+ to Sentinel-2 provided once it is available for all S2 data
    applyCloudScore = True, # We use this since fMask struggles for cloud masking
    applyTDOM = True, # We use TDOM for Landsat since the fMask cloud shadow masking method struggles a lot
    applyFmaskCloudMask = True,
    applyFmaskCloudShadowMask = True,
    applyFmaskSnowMask = False,
    cloudScoreThresh = 10,# The threshold applied to the cloudScore for Landsat and/or Sentinel-2
    performCloudScoreOffset = False,# Set this to False in areas with limited data and/or limited bright cool surfaces
    cloudScorePctl = 10, # If performCloudScoreOffset is set to True, this is the percentile of the cloudScores used to see if an area typically has a high cloudScore
    zScoreThresh = -1, # The TDOM Z-score threshold for finding dark outliers
    shadowSumThresh = 0.35,# The sum of the shadowSumBands has to be below this to be considered dark
    contractPixels = 1.5,# Inward buffer pixel radius to get rid of salt and pepper
    dilatePixels = 3.5, # Outwoard buffer pixel radius to dilate the cloud shadow mask by

    # Image processing
    correctIllumination = False, # Method for illuminating hill shadows. This is largely deprecated and not used
    correctScale = 250,

    # Export parameters
   # exportComposites = False,# Whether to export resulting composites exportCompositeCollection
   # exportPathRoot = 'projects/disco-glass-413820/assets/users/lcms-training_module-2_composites', #'content/drive/MyDrive/lcms-training_module-2_composites',used this i think #'projects/resolute-future-412019/assets/users/lcms-training_module-2_composites', og directory #'projects/earthengine-public/assets/content/drive/MyDrive/lcms-training_module-2_composites',
    exportComposites = False,
    #outputName = 'Landsat-UTKA',
    
    crs = 'EPSG:4326',
    #transform = getImagesLib.common_projections['NLCD_CONUS']['transform'],
    transform = None)
 #  scale = None,
  #  overwrite = True,# Whether to overwrite existing composites

    #Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#clear map
Map.clearMap()
#Turn off layers from previous iteration
Map.turnOffAllLayers()
startYear = 2015
endYear = 2020
#Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'UTKA{}'.format(year),'True')

Map.turnOnInspector()
Map.view()
print('Done')

Get Processed Landsat: 
Start date: Jan 01 2015 , End date: Dec 30 2020
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Defringing L4 and L5
Applying scale factors for C2 L8 data
Only including SLC On Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Cloud Score
Not computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Adding layer: UTKA2015
Adding layer: UTKA2016
Adding layer: UTKA2017
Adding layer: UTKA2018
Adding layer: UTKA2019
Adding layer: UTKA2020
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1992/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://localhost:1992/geeView/?projectID=None&accessToken=ya29.a0AfB_byAyWp6rkD4xmBLlR8urwaVRe0ouYFHcmwEAFSXKAVsebz8qeS8vI

Done


::1 - - [21/Feb/2024 15:03:47] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byAyWp6rkD4xmBLlR8urwaVRe0ouYFHcmwEAFSXKAVsebz8qeS8vI_-yYgGO4tz9jRNX0FJNmuUA8USOk_xP4qMyLfgf9Lg9PSLbyY7v5qv21I1Vfnh6APCCekadOwe4FsU2SmEOIy7mE0Gr4ctTl05HqU9I4JZDmwaCgYKAb8SARISFQHGX2MijOpkT6F953sRBkzMmGTWRQ0173 HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:47] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -


In [8]:
clipped_collection = processedComposites.map(lambda image: image.clip(studyArea))

In [10]:
#clear map
Map.clearMap()
#Turn off layers from previous iteration
Map.turnOffAllLayers()
#Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = clipped_collection.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'UTKA{}'.format(year),'True')
Map.turnOnInspector()
Map.view()
print('Done')

Adding layer: UTKA2015
Adding layer: UTKA2016
Adding layer: UTKA2017
Adding layer: UTKA2018
Adding layer: UTKA2019
Adding layer: UTKA2020
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1992/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://localhost:1992/geeView/?projectID=None&accessToken=ya29.a0AfB_byB4ZTaHAQ7X48F-9iUoPN3qAZ164q9iDkLB9SyaJaKS362sSV-mc7LW-xM98jdM6SizToX_XD9Y94s6OUDUnLH00ASW-gvlHGztuIu7Lv8e2Q4eQEQfjML1n79ovVV1Zy20kMF2n54ulH-yETHkRXR85LM9J-XXQgaCgYKARoSARISFQHGX2Mi_zp_qLxOLdMmBze9tSYL2g0173


Done


::1 - - [21/Feb/2024 15:07:24] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byB4ZTaHAQ7X48F-9iUoPN3qAZ164q9iDkLB9SyaJaKS362sSV-mc7LW-xM98jdM6SizToX_XD9Y94s6OUDUnLH00ASW-gvlHGztuIu7Lv8e2Q4eQEQfjML1n79ovVV1Zy20kMF2n54ulH-yETHkRXR85LM9J-XXQgaCgYKARoSARISFQHGX2Mi_zp_qLxOLdMmBze9tSYL2g0173 HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:07:24] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -
